In [2]:
from urllib.request import Request, urlopen
from urllib.parse import urlparse, parse_qs
import re, os, json
import pprint

import requests as r   ##(urlib Request and requests are two independent libraries)
from bs4 import BeautifulSoup as bs

import pandas as pd
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
# https://pypi.org/project/youtube-transcript-api/ will return a list of dictionaries
# in Anaconda terminal
conda install conda-forge::youtube-transcript-api

<div class="alert alert-block alert-info">
<b>Extracting Subtitles from YouTube Using Python</b>  
    
@JeffMaraPodcast  
179K subscribers, 1.4K videos  
Welcome to the JeffMara Podcast, your gateway to the spiritual, supernatural and the unexplained.  
Join Jeff as he delves into the depths of the unknown, exploring near death experiences, UFOs, interviewing experts in the paranormal field, and sharing chilling real-life encounters.
</div>

**Getting a list of urls**  
HTML siblings tags could probably be used to achieve this, but in the interest of time, my higher self and I went to the all videos website, right-clicked, and then used Console's Inspect to set up automatic scrolling to load all the movies (the first line of code below). We then allowed pasting the next line of code (since Chrome has disabled pasting code) to display all URLs. Manually select URLs to copy/paste into a file of your choice.  

var scroll = setInterval(function(){ window.scrollBy(0, 1000)}, 1000);  
type in console: allow pasting  
window.clearInterval(scroll); console.clear(); urls = $$('a'); urls.forEach(function(v,i,a){if (v.id=="video-title-link" && v.href){console.log('\t'+new Date().toLocaleDateString()+'\t'+v.title+'\t'+v.href+'\t'+v.__shady_native_innerHTML.match(/aria-label=\"(.+?)\"/g)?.[0].match(/[\d,]+ views/g)[0]+'\t')}});

In [3]:
df = pd.read_csv('C:\\Users\\thesk\\JeffMara.csv', encoding='unicode_escape')
df.head(2)

,Title,URL
0,MASTER Reveals How To ACCESS Your SUPERCONSCIO...,https://www.youtube.com/watch?v=bvaHCBxwMYE&t=...
1,"Artifical Intelligence VS Spirituality, The Fu...",https://www.youtube.com/watch?v=8IaO7kE3YQc


In [5]:
def get_id(url):
    '''
    YouTubeTranscriptApi.get_transcript('NY81b-vhmeU') needs the video_id as arg () not URL
    urlparse – split URL into component pieces
    Parse all URLs with urlib parse_qs to capture ids which follow after ?v=
    
   '''
    qry = parse_qs(urlparse(url).query).get('v')
    if qry:
        return qry[0]

In [6]:
Video_id = []
'''
By using our get_id function get video_ids and make a new column 
 '''
for url in df['URL']:
    Video_id.append(get_id(url))
    
df['Video_id'] = Video_id

In [7]:
df.head(2)

,Title,URL,Video_id
0,MASTER Reveals How To ACCESS Your SUPERCONSCIO...,https://www.youtube.com/watch?v=bvaHCBxwMYE&t=...,bvaHCBxwMYE
1,"Artifical Intelligence VS Spirituality, The Fu...",https://www.youtube.com/watch?v=8IaO7kE3YQc,8IaO7kE3YQc


In [8]:
video_ids = list(df['Video_id'])
'''
Made a list of all ids to loop through and by using YouTubeTranscriptApi.get_transcript(video_id)
and make a new column with scraped subtitles 
 '''
subtitles = []
for video_id in video_ids:
    try:
        subtitles.append(YouTubeTranscriptApi.get_transcript(video_id))
    except Exception:
        subtitles.append([{'duration': 0, 'start': 0, 'text': 'None'}])  ## some videos don't have subtitles, they will have this uniform format dictionary then

df['Subtitles'] = subtitles

In [11]:
df.head(2)

,Title,URL,Video_id,Subtitles
0,MASTER Reveals How To ACCESS Your SUPERCONSCIO...,https://www.youtube.com/watch?v=bvaHCBxwMYE&t=...,bvaHCBxwMYE,"[{'text': '[Music]', 'start': 0.57, 'duration'..."
1,"Artifical Intelligence VS Spirituality, The Fu...",https://www.youtube.com/watch?v=8IaO7kE3YQc,8IaO7kE3YQc,"[{'text': '[Music]', 'start': 0.57, 'duration'..."


In [12]:
pprint.pprint(df.Subtitles[0])

[{'duration': 3.87, 'start': 0.57, 'text': '[Music]'},
 {'duration': 6.401,
  'start': 1.079,
  'text': 'and now from beyond our Dimension this'},
 {'duration': 7.6, 'start': 4.44, 'text': 'is the Jeff Mara'},
 {'duration': 7.159,
  'start': 7.48,
  'text': "podcast here's Jeff my guest is RJ Spina"},
 {'duration': 4.72,
  'start': 12.04,
  'text': 'metaphysical teacher and guide who'},
 {'duration': 5.64, 'start': 14.639, 'text': 'overcame serious'},
 {'duration': 6.04,
  'start': 16.76,
  'text': 'illnesses RJ delves into the concept of'},
 {'duration': 5.721,
  'start': 20.279,
  'text': 'self-mastery and He suggests that'},
 {'duration': 6.36,
  'start': 22.8,
  'text': "spirituality isn't worth anything if it"},
 {'duration': 5.64,
  'start': 26.0,
  'text': "isn't applicable in our day-to-day life"},
 {'duration': 4.36,
  'start': 29.16,
  'text': "RJ wel welcome back it's great to see"},
 {'duration': 4.16,
  'start': 31.64,
  'text': 'you again brother thank you for having'},
 

In [13]:
def merge_text(dict_list):
    '''
    Each Subtitles cell is a list containing many separate dictionaries with keys: duration, start, text
    Merge all 'text' values into one large block and create a column Text
    '''
    return ' '.join([d['text'] for d in dict_list])

df['Text'] = df['Subtitles'].apply(merge_text)

In [14]:
df.head(2)

,Title,URL,Video_id,Subtitles,Text
0,MASTER Reveals How To ACCESS Your SUPERCONSCIO...,https://www.youtube.com/watch?v=bvaHCBxwMYE&t=...,bvaHCBxwMYE,"[{'text': '[Music]', 'start': 0.57, 'duration'...",[Music] and now from beyond our Dimension this...
1,"Artifical Intelligence VS Spirituality, The Fu...",https://www.youtube.com/watch?v=8IaO7kE3YQc,8IaO7kE3YQc,"[{'text': '[Music]', 'start': 0.57, 'duration'...",[Music] and now from beyond our Dimension this...


In [15]:
def find_words(text, search_words):
  """Finds the specified words in the given text.
  Args:
    text: The text to search.
    search_words: A list of words to search for.
  Returns:
    A list of found words.
  """
  found = []
  for word in search_words:
    pattern = r'\b' + re.escape(word) + r'\b'  # Word boundary regex not to find dog in word dogger
    if re.search(pattern, text, re.IGNORECASE):
      found.append(word)
  return found

In [16]:
search_words = ['cat', 'dog', 'pet','cats', 'dogs', 'pets', 'animals']

# Apply the function to create a new column
df['Found_words'] = df['Text'].apply(lambda x: find_words(x, search_words))
df.head(4)

In [19]:
def find_and_extract(text_list, search_words, key_to_search, key_to_extract):
  """Finds specified words in a list of dictionaries, extracting timestamp values from found words' dictionaries.
  Args:
    text_list: A list of dictionaries to search.
    search_words: A list of words to search for.
    key_to_search: The key to search for the words.
    key_to_extract: The key to extract the value from if a match is found.
  Returns:
    A list of extracted values.
  """
  found = []
  for item in text_list:
    for word in search_words:
      pattern = r'\b' + re.escape(word) + r'\b'
      if re.search(pattern, item[key_to_search], re.IGNORECASE):
        found.append(item[key_to_extract])
  return found

In [21]:
# Target words and keys
search_words = ['cat', 'dog', 'pet','cats', 'dogs', 'pets', 'animals']
key_to_search = 'text'
key_to_extract = 'start'

# Apply the function to create a new column
df['Timestamps'] = df['Subtitles'].apply(lambda x: find_and_extract(x, search_words, key_to_search, key_to_extract))
df.head(4)

,Title,URL,Video_id,Subtitles,Text,Found_words,Timestamps
0,MASTER Reveals How To ACCESS Your SUPERCONSCIO...,https://www.youtube.com/watch?v=bvaHCBxwMYE&t=...,bvaHCBxwMYE,"[{'text': '[Music]', 'start': 0.57, 'duration'...",[Music] and now from beyond our Dimension this...,[],[]
1,"Artifical Intelligence VS Spirituality, The Fu...",https://www.youtube.com/watch?v=8IaO7kE3YQc,8IaO7kE3YQc,"[{'text': '[Music]', 'start': 0.57, 'duration'...",[Music] and now from beyond our Dimension this...,"[dog, cats]","[1752.0, 1754.08, 1760.279]"
2,Man Crosses Over & Sees AI Souls - Near Death ...,https://www.youtube.com/watch?v=vzPrxUvA1N4,vzPrxUvA1N4,"[{'text': '[Music]', 'start': 0.57, 'duration'...",[Music] and now from beyond our Dimension this...,"[cat, dog, animals]","[537.6, 2214.839, 2217.72, 2226.88, 2702.2]"
3,Man Who Was FACE To FACE With An ET Inside A Box,https://www.youtube.com/watch?v=lAINwUxII1A&t=...,lAINwUxII1A,[{'text': 'welcome to beyond the stars with Ti...,welcome to beyond the stars with Tim and [Musi...,[],[]


In [22]:
def find_and_return_dict(text_list, search_words, key_to_search):
  """Finds specified words in a list of dictionaries and returns the entire dictionary (not just the timestamp).
  Args:
    text_list: A list of dictionaries to search.
    search_words: A list of words to search for.
    key_to_search: The key to search for the words.
  Returns:
    A list of dictionaries where a match was found.
  """
  found_dicts = []
  for item in text_list:
    for word in search_words:
      pattern = r'\b' + re.escape(word) + r'\b'
      if re.search(pattern, item[key_to_search], re.IGNORECASE):
        found_dicts.append(item)
  return found_dicts

In [23]:
search_words = ['cat', 'dog', 'pet','cats', 'dogs', 'pets', 'animals']
key_to_search = 'text'

df['Found_dicts'] = df['Subtitles'].apply(lambda x: find_and_return_dict(x, search_words, key_to_search))
df.head(4)

,Title,URL,Video_id,Subtitles,Text,Found_words,Timestamps,Found_dicts
0,MASTER Reveals How To ACCESS Your SUPERCONSCIO...,https://www.youtube.com/watch?v=bvaHCBxwMYE&t=...,bvaHCBxwMYE,"[{'text': '[Music]', 'start': 0.57, 'duration'...",[Music] and now from beyond our Dimension this...,[],[],[]
1,"Artifical Intelligence VS Spirituality, The Fu...",https://www.youtube.com/watch?v=8IaO7kE3YQc,8IaO7kE3YQc,"[{'text': '[Music]', 'start': 0.57, 'duration'...",[Music] and now from beyond our Dimension this...,"[dog, cats]","[1752.0, 1754.08, 1760.279]",[{'text': 'audience are probably aware that ca...
2,Man Crosses Over & Sees AI Souls - Near Death ...,https://www.youtube.com/watch?v=vzPrxUvA1N4,vzPrxUvA1N4,"[{'text': '[Music]', 'start': 0.57, 'duration'...",[Music] and now from beyond our Dimension this...,"[cat, dog, animals]","[537.6, 2214.839, 2217.72, 2226.88, 2702.2]","[{'text': 'when it was flowing over your dog',..."
3,Man Who Was FACE To FACE With An ET Inside A Box,https://www.youtube.com/watch?v=lAINwUxII1A&t=...,lAINwUxII1A,[{'text': 'welcome to beyond the stars with Ti...,welcome to beyond the stars with Tim and [Musi...,[],[],[]


In [24]:
def find_and_combine(text_list, search_words, key_to_search):
  """Finds specified words in a list of dictionaries, returns original dict and combined text values from next 15 dictionaries.
  Args:
    text_list: A list of dictionaries to search.
    search_words: A list of words to search for.
    key_to_search: The key to search for the words.
  Returns:
    A list of dictionaries containing the found dictionary and combined text.
  """
  found_dicts = []
  for i, item in enumerate(text_list):
    for word in search_words:
      pattern = r'\b' + re.escape(word) + r'\b'
      if re.search(pattern, item[key_to_search], re.IGNORECASE):
        found_dict = item.copy()  # Create a copy to avoid modifying original data
        fragment = ' '.join(d['text'] for d in text_list[i:i+16] if 'text' in d)
        found_dict['FRAGMENT'] = fragment
        found_dicts.append(found_dict)
  return found_dicts

In [25]:
search_words = ['cat', 'dog', 'pet','cats', 'dogs', 'pets', 'animals']
key_to_search = 'text'

df['Fragment'] = df['Subtitles'].apply(lambda x: find_and_combine(x, search_words, key_to_search))
df.head(4)

,Title,URL,Video_id,Subtitles,Text,Found_words,Timestamps,Found_dicts,Fragment
0,MASTER Reveals How To ACCESS Your SUPERCONSCIO...,https://www.youtube.com/watch?v=bvaHCBxwMYE&t=...,bvaHCBxwMYE,"[{'text': '[Music]', 'start': 0.57, 'duration'...",[Music] and now from beyond our Dimension this...,[],[],[],[]
1,"Artifical Intelligence VS Spirituality, The Fu...",https://www.youtube.com/watch?v=8IaO7kE3YQc,8IaO7kE3YQc,"[{'text': '[Music]', 'start': 0.57, 'duration'...",[Music] and now from beyond our Dimension this...,"[dog, cats]","[1752.0, 1754.08, 1760.279]",[{'text': 'audience are probably aware that ca...,[{'text': 'audience are probably aware that ca...
2,Man Crosses Over & Sees AI Souls - Near Death ...,https://www.youtube.com/watch?v=vzPrxUvA1N4,vzPrxUvA1N4,"[{'text': '[Music]', 'start': 0.57, 'duration'...",[Music] and now from beyond our Dimension this...,"[cat, dog, animals]","[537.6, 2214.839, 2217.72, 2226.88, 2702.2]","[{'text': 'when it was flowing over your dog',...","[{'text': 'when it was flowing over your dog',..."
3,Man Who Was FACE To FACE With An ET Inside A Box,https://www.youtube.com/watch?v=lAINwUxII1A&t=...,lAINwUxII1A,[{'text': 'welcome to beyond the stars with Ti...,welcome to beyond the stars with Tim and [Musi...,[],[],[],[]


In [30]:
df.at[40, 'Fragment']

[{'text': 'of deceased loved ones and pets and',
  'start': 54.16,
  'duration': 5.239,
  'FRAGMENT': "of deceased loved ones and pets and Angels lots of angels as we're transition and uh everybody can see them when everybody is dying University based research shows that 90% of people at the end of their lives can see the spirits of deceased loved ones and pets and it used to be je that we would think oh Grandma's just hallucinating when she's saying she's seeing her father who's been dead for 80 years well no she's not she's actually seeing him seeing his spirit so the phases of of transition as I call them are different configurations that these angels and spirits of loved"},
 {'text': 'of deceased loved ones and pets and it',
  'start': 72.52,
  'duration': 4.76,
  'FRAGMENT': "of deceased loved ones and pets and it used to be je that we would think oh Grandma's just hallucinating when she's saying she's seeing her father who's been dead for 80 years well no she's not she's actually

In [29]:
# Just saving to Excel gives warning that it exceeds Excel's limit for URLS
# Create an ExcelWriter object with the option not to convert strings to URLS
writer = pd.ExcelWriter(r'C:\Users\thesk\Desktop\exported_Youtube.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df.to_excel(writer)
writer.close()

C:\Users\thesk\AppData\Local\Temp\ipykernel_3716\2216423509.py:3: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter(r'C:\Users\thesk\Desktop\exported_Youtube.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})


In conclusion, there don't seem to be any downloaded subtitles for my previously manually timestamped and stored videos including NDEs and pets in afterlife.  
Maybe I should try a different scraper to see if I can get more captions downloaded. The ones I downloaded don't really match what I was searching for, despite having numerous animal references.  With this code, yYou may look up words like soul contracts, past lifetimes, and so on.